In [152]:
# Use centrailzed training to compare with federated learning
epochs = 30
n_train_items = 6000

In [153]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import numpy as np


In [155]:
class Arguments():
    def __init__(self):
        self.batch_size = 60
        self.test_batch_size = 60
        self.epochs = epochs
        self.lr = 0.02
        self.momentum = 0
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [156]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

train_dataloader = [
        ((data), (target))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
# test_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(test_loader)
#         if i < n_train_items / args.batch_size
#     ]


In [157]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, 1)
        self.conv2 = nn.Conv2d(32, 64, 5, 1)
        self.fc1 = nn.Linear(3136, 512)
        self.fc2 = nn.Linear(512, 10)
        self.same_padding = nn.ReflectionPad2d(2)
#         self.dropout05 = nn.Dropout2d(0.2)
#         self.dropout03 = nn.Dropout2d(0.2)
#         self.batch32 = nn.BatchNorm2d(32)
#         self.batch64 = nn.BatchNorm2d(64)
        
    def forward(self, x):
        x = self.same_padding(x)
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
#         x = self.batch32(x)
#         x = self.dropout03(x)
        x = self.same_padding(x)
        x = F.relu(self.conv2(x))
#         x = self.batch64(x)
#         x = self.dropout05(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 3136)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [158]:
def train(args, device, model, train_dataloader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_dataloader) * args.batch_size,
                100. * batch_idx / len(train_dataloader), loss.item()))

In [159]:
def test(args, device, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    target_len = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()
            target_len += len(target.view_as(pred))
    test_loss /= len(test_loader)*(args.batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader)* (args.batch_size),
        100. * correct / target_len))

In [160]:
model =  Net()
print(sum(p.numel() for p in model.parameters()))

1663370


In [161]:
#%%time
model = Net().to(device)
optimizer =  optim.SGD(model.parameters(), lr=args.lr)
for epoch in range(1, args.epochs + 1):
    train(args, device, model, train_dataloader, optimizer, epoch)
    test(args, device, model, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/6000 (0%)]	Loss: 2.289546
Train Epoch: 1 [1800/6000 (30%)]	Loss: 1.771738
Train Epoch: 1 [3600/6000 (60%)]	Loss: 0.712529
Train Epoch: 1 [5400/6000 (90%)]	Loss: 0.471051

Test set: Average loss: 0.5182, Accuracy: 8328/10020 (83.28%)

Train Epoch: 2 [0/6000 (0%)]	Loss: 0.640840
Train Epoch: 2 [1800/6000 (30%)]	Loss: 0.447032
Train Epoch: 2 [3600/6000 (60%)]	Loss: 0.155297
Train Epoch: 2 [5400/6000 (90%)]	Loss: 0.191190

Test set: Average loss: 0.2755, Accuracy: 9189/10020 (91.89%)

Train Epoch: 3 [0/6000 (0%)]	Loss: 0.324951
Train Epoch: 3 [1800/6000 (30%)]	Loss: 0.251916
Train Epoch: 3 [3600/6000 (60%)]	Loss: 0.096245
Train Epoch: 3 [5400/6000 (90%)]	Loss: 0.127122

Test set: Average loss: 0.2087, Accuracy: 9372/10020 (93.72%)

Train Epoch: 4 [0/6000 (0%)]	Loss: 0.204915
Train Epoch: 4 [1800/6000 (30%)]	Loss: 0.165191
Train Epoch: 4 [3600/6000 (60%)]	Loss: 0.070396
Train Epoch: 4 [5400/6000 (90%)]	Loss: 0.104204

Test set: Average loss: 0.1715, Accuracy: 9473/10020 (9